In [1]:
########################### Libraries ####################################
import astropy
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import json
import scipy
from scipy.signal import argrelextrema
import matplotlib.backends.backend_pdf
from gatspy import datasets, periodic
from scipy.signal import find_peaks
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import average_precision_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
from sklearn.metrics import accuracy_score
import timeit
from astropy.timeseries import LombScargle
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss  
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
import requests
import urllib
import os
from os import path

In [2]:
data_dir = os.getcwd()
directory = "catalogs"
subdirectoryres='res'
path = os.path.join(data_dir, directory)
pathres= os.path.join(path, subdirectoryres) 

### df1 - homogenuous set in zg , df2 - homogenuous set in zr, df3 - overall (zg(min)+zr(min))

In [3]:
# df1=pd.read_csv('df1_homogen_zg.csv')
# df2=pd.read_csv('df1_homogen_zr.csv')
# df3=pd.read_csv('df1_homogen_summ.csv')


# NEW:

df1=pd.read_csv('homogen_zg_example.csv', sep='\t')
df2=pd.read_csv('homogen_zr_example.csv', sep='\t')
df3=pd.read_csv('homogen_summ_example.csv', sep='\t')


# df_zg.to_csv("df1_homogen_zg.csv", sep='\t')
# df_zr.to_csv("df1_homogen_zr.csv", sep='\t')
# df_summ.to_csv("df1_homogen_summ.csv", sep='\t')

In [4]:
# df1=df1[df1['Nobs']>30]
# df1=df1.iloc[:10000]
# df1=df1.reset_index()

# df2=df2[df2['Nobs']>30]
# df2=df2.iloc[:10000]
# df2=df2.reset_index()


# df3=df3[df3['Nobs']>30]
# df3=df3.iloc[:10000]
# df3=df3.reset_index()

In [5]:
# df2[df2['target']==1].shape[0]

### Features - features generation, df- homogenuous df with 'Name' columns (names of files in the folder)

### Metrics - function to receive final results

In [6]:
def Features(var_type,df):
    start = timeit.default_timer()
    df['period-0']=0.
    df['power-0']=0.
    df['period-0']=0.
    df['mean_magn']=0.
    df['amplitude']=0.
    df['trand_angle']=0.
    df['skew']=0.
    df['std']=0.
    df['target']=0
#     df=df.iloc[:n]
    names=df['Name'].tolist()
    for i in range(df.shape[0]):
        if df['Type'][i]==var_type:
            df['target'][i]=1
        else:
            df['target'][i]=0
    # model = periodic.LombScargleAstroML(fit_period=False)    
    # model.optimizer.set(period_range=(1, 70), first_pass_coverage=30)    
    
    k=0
    for item in names:
        try:
            with open(pathres+"\\{}.txt".format(item)) as file:
                res = file.read().replace("'", '"')
                res=json.loads(res)
                obj_mjd=[]
                obj_mag=[]
                obj_magerr=[]

                for obj in res.keys():
                    for ii in res[obj]['lc']:
                        obj_mjd.append(ii['mjd'])
                        obj_mag.append(ii['mag'])
                        obj_magerr.append(ii['magerr'])

                    frequency, power = LombScargle(obj_mjd, obj_mag, obj_magerr).autopower()
                    df['period-0'][k]=1/frequency[np.argmax(power)]
                    df['power-0'][k]=max(power)
                    df['mean_magn'][k] = np.array(obj_mag).mean()
                    df['amplitude'][k] = (np.array(obj_mag).max() - np.array(obj_mag).min())/2
                    df['trand_angle'][k] = np.polyfit(np.array(obj_mjd), np.array(obj_mag), 1)[0] 
                    df['skew'][k] = scipy.stats.skew(np.array(obj_mag))
                    df['std'][k] = np.array(obj_mag).std()
                    k=k+1
        except:
            print('error')
    return df                
    #           indices = find_peaks(power)
    #           model.fit(obj_mjd, obj_mag, obj_magerr)
    #           for i in range(3):
    #               df['period-'+str(i)][k]=np.round(model.find_best_periods(n_periods=5,return_scores=True)[0][i],3)
    #                   df['power-'+str(i)][k]=power[int(indices[0][i])]

#         except:
#             print(1)



def Metrics(df):
    n=sum(df['target'].tolist())
    dff=df[df['target']==0].iloc[:n]
    df=df[df['target']==1]
    r=[df,dff]
    df=pd.concat(r)
    print(df.shape)
    
    X = np.array(df[['period-0','power-0','mean_magn','amplitude']])
    Y = np.array(df['target'])
    validation_size = 0.3
    seed = 2
    X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=seed)
    model = XGBClassifier(random_state=34, seed=7, colsample_bytree=0.8, subsample=0.6, n_estimators=700)
    model.fit(X_train,Y_train)
    pred = model.predict(X_validation)
    Metrics=pd.DataFrame(columns=['Accuracy','precision','Roc','F1'],index=range(1))
    Metrics['Roc'][0]=roc_auc_score(pred.tolist(), Y_validation.tolist())
    Metrics['Accuracy'][0]=accuracy_score(Y_validation.tolist(), pred.tolist())
    Metrics['precision'][0]=precision_score(Y_validation.tolist(), pred.tolist())
    Metrics['F1'][0]=f1_score(Y_validation.tolist(), pred.tolist())
    return Metrics

In [7]:
def Metrics_classic(df):
    print('N(type1)=',len(df[df['target']==1]))
    print('N(type2)=',len(df[df['target']==0]))
    print('overall=',df.shape[0])
    print('best_random_res=',max(len(df[df['target']==1]),len(df[df['target']==0]))/df.shape[0] )
    X = np.array(df[['period-0','power-0','mean_magn','amplitude']])
    Y = np.array(df['target'])
    validation_size = 0.3
    seed = 2
    X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=seed)
    model = XGBClassifier(random_state=34, seed=7, colsample_bytree=0.8, subsample=0.6, n_estimators=700)
    model.fit(X_train,Y_train)
    pred = model.predict(X_validation)
    Metrics=pd.DataFrame(columns=['Accuracy','precision','Roc','F1'],index=range(1))
    Metrics['Roc'][0]=roc_auc_score(pred.tolist(), Y_validation.tolist())
    Metrics['Accuracy'][0]=accuracy_score(Y_validation.tolist(), pred.tolist())
    Metrics['precision'][0]=precision_score(Y_validation.tolist(), pred.tolist())
    Metrics['F1'][0]=f1_score(Y_validation.tolist(), pred.tolist())
    return Metrics

In [8]:
#Cepheids 'zg' vs cepheids 'zr'
def compare(var_type,df1,df2):
    start = timeit.default_timer()
    n=len(df1[df1['Type']==var_type])+len(df2[df2['Type']==var_type])
    global df
    df=pd.DataFrame(columns=['ra','dec','Nobs','filter','Name'],index=range(n))
    df['period-0']=0.
    df['power-0']=0.
    df['period-0']=0.
    df['mean_magn']=0.
    df['amplitude']=0.
    df['trand_angle']=0.
    df['skew']=0.
    df['std']=0.
    df['target']=0
#     df=df.iloc[:n]

    names1=df1['Name'].tolist()
    names2=df2['Name'].tolist()
    k=0
    
    for i in range(df1.shape[0]):
        if df1['Type'][i]==var_type:
            df['target'][k]=0
            df['Name'][k]=df1['Name'][i]
            df['filter'][k]='zg'
            df['ra'][k]=df1['ra'][i]
            df['dec'][k]=df1['dec_x'][i]
            df['Nobs'][k]=df1['Nobs'][i]
            k=k+1
        
    for i in range(df2.shape[0]):  
        if df2['Type'][i]==var_type:
            df['target'][k]=1
            df['Name'][k]=df2['Name'][i]
            df['filter'][k]='zr'
            df['ra'][k]=df2['ra'][i]
            df['dec'][k]=df2['dec_x'][i]
            df['Nobs'][k]=df2['Nobs'][i]
            k=k+1

    names=df[df['Name'].notna()==True]['Name'].tolist()
    k=0
    for item in names:
        try:
            with open(r"C:\Users\rtut6\Desktop\Backup\res\{}.txt".format(item)) as file:
                res = file.read().replace("'", '"')
                res=json.loads(res)
                obj_mjd=[]
                obj_mag=[]
                obj_magerr=[]

                for obj in res.keys():
                    for ii in res[obj]['lc']:
                        obj_mjd.append(ii['mjd'])
                        obj_mag.append(ii['mag'])
                        obj_magerr.append(ii['magerr'])

                    frequency, power = LombScargle(obj_mjd, obj_mag, obj_magerr).autopower()
                    df['period-0'][k]=1/frequency[np.argmax(power)]
                    df['power-0'][k]=max(power)
                    df['mean_magn'][k] = np.array(obj_mag).mean()
                    df['amplitude'][k] = (np.array(obj_mag).max() - np.array(obj_mag).min())/2
                    df['trand_angle'][k] = np.polyfit(np.array(obj_mjd), np.array(obj_mag), 1)[0] 
                    df['skew'][k] = scipy.stats.skew(np.array(obj_mag))
                    df['std'][k] = np.array(obj_mag).std()
                    k=k+1
        except:
            print(1)
            
    return  df 

In [9]:
df1

,Unnamed: 0,ra,dec_x,Nobs,filter,Name,coord,Type,GCVS
0,0,12.847,34.37674,44,zg,10,005123.32 +342236.8,M,RR And
1,1,344.524,48.75930,49,zg,104,225805.76 +484533.7,SR,CS And
2,2,344.905,51.67148,134,zg,105,225937.28 +514016.7,LB,CT And
3,3,351.161,49.05054,46,zg,116,232438.58 +490302.1,SR,DH And
4,4,351.735,48.95586,46,zg,117,232656.34 +485721.3,CWB,DI And *
...,...,...,...,...,...,...,...,...,...
257,257,33.273,40.82535,34,zg,467,021305.43 +404931.1,EW,V0568 And
258,258,345.857,44.20377,45,zg,487,230325.74 +441213.6,EA,V0590 And
259,259,345.857,44.20377,49,zg,487,230325.74 +441213.6,EA,V0590 And
260,260,346.813,48.47815,49,zg,500,230715.06 +482841.3,LB,V0603 And


In [10]:
df1=Features('M',df1)
df2=Features('M',df2)
df3=Features('M',df3)

C:\Users\rtut6\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\rtut6\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\rtut6\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\rtut6\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: Settin

error


In [11]:
df2

,Unnamed: 0,ra,dec_x,Nobs,filter,Name,coord,Type,GCVS,period-0,power-0,mean_magn,amplitude,trand_angle,skew,std,target
0,0,18.884,37.62657,57,zr,162,011532.09 +373734.2,UGSU,FO And *,76.604456,0.299072,16.472857,2.2525,-0.006162,-0.328402,1.375817,0
1,1,29.752,36.21184,30,zr,165,015900.41 +361242.4,RR,FR And,3.060783,1.003059,16.519429,0.4150,0.001024,-0.137024,0.243824,0
2,2,10.680,41.26829,28,zr,2,004243.1 +411605.0,SNI,S And *,55.060177,0.438238,14.676565,0.3250,0.000800,-0.324860,0.161997,0
3,3,11.251,43.25581,35,zr,207,004500.20 +431520.8,RRAB,IK And *,125.211397,0.620207,16.806871,0.3955,-0.000653,-0.465605,0.234472,0
4,4,13.888,38.36757,35,zr,212,005533.07 +382203.7,RRAB,IQ And,17.591684,0.484564,16.439000,0.5020,-0.000658,-0.410284,0.289735,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,179,33.273,40.82534,44,zr,467,021305.43 +404931.1,EW,V0568 And,50.851199,0.470953,13.846437,0.2300,-0.000820,0.243274,0.157406,0
180,180,345.857,44.20376,54,zr,487,230325.74 +441213.6,EA,V0590 And,1.019702,0.171996,13.683406,0.3650,-0.000066,0.339705,0.213836,0
181,181,345.857,44.20376,56,zr,487,230325.74 +441213.6,EA,V0590 And,6.194339,0.574450,13.720681,0.1960,0.000003,-0.154939,0.128113,0
182,182,346.813,48.47814,58,zr,500,230715.06 +482841.3,LB,V0603 And,1.192077,0.272449,13.526950,0.3580,-0.000227,0.094652,0.222212,0


In [12]:
M1=Metrics(df1)
M2=Metrics(df2)
M3=Metrics(df2)
r=[M1,M2,M3]
m=pd.concat(r)

(76, 17)
(18, 17)
(18, 17)


In [13]:
m.index=['zg','zr','overall']

In [14]:
m

,Accuracy,precision,Roc,F1
zg,0.869565,0.9,0.873077,0.857143
zr,0.666667,0.666667,0.666667,0.666667
overall,0.666667,0.666667,0.666667,0.666667


In [15]:
# compare('DCEP',df1,df2)

In [16]:
# Metrics_classic(df)

In [17]:
# model = periodic.LombScargleAstroML(fit_period=False)
# model.optimizer.set(period_range=(1, 70), first_pass_coverage=90)
# model.fit(obj_mjd, obj_mag, obj_magerr)
# model.find_best_periods(n_periods=5,return_scores=True)
# model.periodogram(periods=True)
# periods = np.linspace(0.1, 10, 100)

# omega=2*np.pi/periods
# plt.scatter(scipy.signal.lombscargle(obj_mjd, obj_mag, omega), periods)

In [18]:
# from astropy.timeseries import LombScargle
# frequency, power = LombScargle(obj_mjd, obj_mag, obj_magerr).autopower()
# frequency[np.argmax(power)]

# # plt.ylim(-1, 5)
# plt.plot(frequency, power)
# plt.ylabel('power')
# plt.xlabel('frequency cyclic')
# ax = plt.gca()
# ax.set_ylim(0,1)
# plt.title(obj)
# ax.scale('log')
# plt.ylim(-0.2, 0.6)
# plt.title('')
# print(1/frequency[np.argmax(power)])
# print(1/0.78)
# print(1/1.2)
# print(1/0.41)

In [19]:
# from scipy.signal import find_peaks

# indices = find_peaks(power, threshold=0.001)

In [20]:
# indices[0]

In [21]:
# from gatspy.periodic import LombScargleFast
# model = LombScargleFast().fit(obj_mjd, obj_mag, obj_magerr)
# period, power = model.periodogram_auto(nyquist_factor=200)

# #                     D1=np.max(power[peak_data[0]])
# #                     ind1=power[peak_data[0]].argmax()
# #                     power[peak_data[0][ind1]]=0
# #                     D2=np.max(power[peak_data[0]])
# #                     ind2=power[peak_data[0]].argmax() 
# #                     object_return['max_fs'] =0

# Fs= 2*np.pi/(period[power.argmax()])
# Fs1= 2*np.pi/(period[peak_data[0][ind2]])

#### Cumulative curves for Nobs


In [22]:
# # names=df1['Type'].value_counts()[:100].index.tolist()
# names=['DCEP','CEP','BCEP','UG','DCEPS','CEP(B)','CEPS','RR']
# # pdf = matplotlib.backends.backend_pdf.PdfPages("234.pdf")
# for item in names:
    
#     data1=df1[df1['Type']==item]['Nobs'].tolist()
#     data2=df2[df2['Type']==item]['Nobs'].tolist()
#     data3=df3[df3['Type']==item]['Nobs'].tolist()
    
#     values1, base1 = np.histogram(data1, bins=60)
#     values2, base2 = np.histogram(data2, bins=60)
#     values3, base3 = np.histogram(data3, bins=60)
    
#     cumulative1 = np.cumsum(values1)
#     cumulative2 = np.cumsum(values2)
#     cumulative3 = np.cumsum(values3)
    
#     #visualisation + saving
    
#     f=plt.figure(figsize=(8,5))
#     plt.gca().set_xlim([0,160])
#     ax = plt.gca()
# #     ax.set_ylim([0,2500])
#     ax.set_facecolor((1.0, 1, 1))
#     plt.plot(base1[:-1], len(data1)-cumulative1, c='green')
#     plt.plot(base2[:-1], len(data2)-cumulative2, c='red')
#     plt.plot(base3[:-1], len(data3)-cumulative3, c='blue',)
#     plt.legend(['ZG','ZR','SUMM'])
#     plt.title(item)
#     plt.grid(True,linestyle="--")

# #     pdf.savefig(f) #saving pdf
#     plt.show()
    
# # pdf.close()
